In [1]:
import macropy.activate
import importlib
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code.iff_macro import set_config
from Code.lstm import lstmPolicyPredictor
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy, PassiveEnv
from Code.SNN import RSNN, FeedForwardSNN, TestNN, newRSNN

In [2]:
'''
config = {
    'ALPHA': 0,
    'BETA': 1,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss',
    'SIM_TIME': 1
}
'''
config = {
    'ALPHA': 0.7,
    'BETA': 0.95,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss',
    'SIM_TIME': 1
}

set_config(config)
import Code.macroNeurons as Neurons
importlib.reload(Neurons)

<module 'Code.macroNeurons' from '../Code/macroNeurons.py'>

In [3]:
env = MultiMountainCar()

In [4]:
BATCH_SIZE = 64

In [5]:
#model = lstmPolicyPredictor(1,8,16)
#model = RSNN(config, 1, 32, 16, 1, Neurons.LIFNeuron, Neurons.AdaptiveNeuron, Neurons.OutputNeuron)
#model = FeedForwardSNN(config, [1, 128, 128, 1], Neurons.LIFNeuron, Neurons.OutputNeuron)
#model = TestNN(config, [1, 128, 10, 10, 1], Neurons.LIFNeuron, Neurons.OutputNeuron)
#model = newRSNN(config, 1, 64, 32, 64, 128, 1, Neurons.LIFNeuron, Neurons.CooldownNeuron, Neurons.OutputNeuron)
model = newRSNN(config, 1, 64, 32, 64, 128, 1, Neurons.CooldownNeuron, Neurons.CooldownNeuron, Neurons.OutputNeuron)

teacher = LookupPolicy()

In [6]:
import gym
testenv = gym.make('MountainCar-v0')

def validate(num_runs):
    sum = 0
    for i in range(num_runs):
        obs = testenv.reset()
        state = None
        for t in range(300):
            output, state = model(torch.tensor([[[obs[0]]]], dtype=torch.float), state)
            action = 2 if output > 0.5 else 0
            obs, _, done, _ = testenv.step(action)
            if done:
                #print(t+1)
                sum += t + 1
                break
    print('Validation: ', sum/num_runs)


/opt/conda/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
bce = nn.BCELoss(reduction='none') #reduction='sum'
optimizer = optim.Adam(model.parameters(), lr=0.00002)#0.00001

In [8]:
#torch.autograd.set_detect_anomaly(True)

In [9]:
for i in range(5000):
    model.zero_grad()
    observation = env.reset(BATCH_SIZE)
    state = None
    loss = 0
    for t in range(200):
        output, state = model(observation[:,:1].unsqueeze(0), state)
        target = teacher(observation)/2
        #print(observation[:,:1].unsqueeze(0).shape, output.shape, target.shape)
        action = (output.squeeze() > 0.5) * 2.0
        observation, _, done, _ = env.step(action)
        loss = loss + (bce(torch.sigmoid(output.squeeze()), target) * (~done).float()).mean()
        #print(t, loss)
        if done.all():
            break
    if i%10 == 0:
        print(loss.item(), t+1, i) #, ((outputs>0.5) != targets).sum()
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        validate(10)
    

147.7621612548828 200 0
Validation:  200.0
148.1138916015625 200 10
146.549072265625 200 20
145.01268005371094 200 30
145.22984313964844 200 40
145.2532958984375 200 50
144.9879913330078 200 60
144.16697692871094 200 70
143.80499267578125 200 80
143.70643615722656 200 90
142.8672332763672 200 100
Validation:  200.0
143.32290649414062 200 110
142.17941284179688 200 120
142.15655517578125 200 130
141.6949462890625 200 140
141.9780731201172 200 150
141.13101196289062 200 160
140.79229736328125 200 170
140.04774475097656 200 180
139.76321411132812 200 190
139.09693908691406 200 200
Validation:  200.0
138.5808563232422 200 210
138.35226440429688 200 220
137.50079345703125 200 230
136.7863006591797 200 240
135.95069885253906 200 250
134.58200073242188 200 260
133.463134765625 200 270
132.14328002929688 200 280
131.81033325195312 200 290
130.37530517578125 200 300
Validation:  200.0
129.76202392578125 200 310
128.9149169921875 200 320
127.6979751586914 200 330
126.88092803955078 200 340
125.9

KeyboardInterrupt: 

In [ ]:
validate(10)

In [ ]:
teacher(observation)/2



In [ ]:
observation

In [ ]:
#torch.save(model, '../models/rsnn_mountaincar2')




In [ ]:
model.input_linear.bias
